In [6]:
import numpy as np
import mbore
import tqdm.notebook
import os

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# set up the things we need to load all optimisation runs
problem_names = ["DTLZ", "WFG", "RW", "WFG_HD"]
processed_results_dir = "processed_results"

final_results_dir = "final_results"

# note that these are the names used to run the code (not in the paper)
models_and_optimizers = [
    ("XGB", "CMAES"),
    ("FCNet", "Grad"), # known as 'MLP' in the paper
    ("GP", "EI"),
]

model_dict = {
    "XGB": "CMAES",
    "FCNet": "Grad",
    "GP": "EI"
}

scalarizers = [
    "HypI",
    "DomRank",
    "HypCont", # known as 'PHC' in the paper
    "ParEGO", # known as 'AT' in the paper
]

# settings that we used for all problems
gamma_start = 0.33
gamma_end = None
gamma_schedule = None
budget = 300

### Combine and save performance indicator and timing information

In [9]:
if not os.path.exists(final_results_dir):
    os.mkdir(final_results_dir)
    print(f"Making directory {final_results_dir}")

for original_problem_name in tqdm.notebook.tqdm(problem_names):
    savename = f"{original_problem_name:s}.npz"
    savepath = os.path.join(final_results_dir, savename)

    if os.path.exists(savepath):
        print(f"Results file already exists, skipping: {savepath:s}")
        continue

    problem_name, prob_dict = mbore.problem_sets.get_problem_dict(
        original_problem_name
    )
    total_problems = mbore.problem_sets.get_number_of_problems(prob_dict)
    print(f"Problem set: {problem_name:s}. {total_problems:d} total problems")

    # load in all the results and create a dictionary structured like:
    #     D[problem_id][dim][fdim][scalarizer][(model, opt)] = {
    #         'igd+': ..., 'hv': ..., 'Xtrs': ..., 'Ytrs': ...
    #     }
    D = {}
    
    total = total_problems * len(models_and_optimizers) * len(scalarizers)

    with tqdm.notebook.tqdm(total=total, leave=False) as pbar:
        for problem_id in prob_dict:
            D[problem_id] = {}

            for dim, fdims in prob_dict[problem_id]:
                D[problem_id][dim] = {}

                for fdim in fdims:
                    D[problem_id][dim][fdim] = {}

                    for scalarizer in scalarizers:
                        D[problem_id][dim][fdim][scalarizer] = {}

                        for model, model_opt_method in model_dict.items():
                            if model == "GP":
                                meth_gamma_start = None
                                meth_gamma_end = None
                                meth_gamma_schedule = None
                            else:
                                meth_gamma_start = gamma_start
                                meth_gamma_end = gamma_end
                                meth_gamma_schedule = gamma_schedule

                            save_path = mbore.util.generate_save_filename(
                                problem_name=problem_name,
                                problem_id=problem_id,
                                dim=dim,
                                fdim=fdim,
                                run_no=None,
                                scalarizer=scalarizer,
                                model=model,
                                model_opt_method=model_opt_method,
                                gamma_start=meth_gamma_start,
                                gamma_end=meth_gamma_end,
                                gamma_schedule=meth_gamma_schedule,
                                save_dir=processed_results_dir,
                            )

                            with np.load(save_path) as fd:
                                # ['Xtrs', 'Ytrs', 'hvs', 'igds', 'timing]
                                # Xtrs = fd["Xtrs"]
                                # Ytrs = fd["Ytrs"]
                                hvs = fd["hvs"]
                                igds = fd["igds"]
                                timing = fd["timing"]

                            D[problem_id][dim][fdim][scalarizer][model] = {
                                # "Xtrs": Xtrs,
                                # "Ytrs": Ytrs,
                                "igd+": igds,
                                "hv": hvs,
                                "timing": timing,
                            }

                            pbar.update()

    
    np.savez(savepath, D=D, original_problem_name=original_problem_name)
    print(f"Saved: {savepath:s}")

  0%|          | 0/4 [00:00<?, ?it/s]

Problem set: DTLZ. 56 total problems


  0%|          | 0/672 [00:00<?, ?it/s]

Saved: final_results\DTLZ.npz
Problem set: WFG. 63 total problems


  0%|          | 0/756 [00:00<?, ?it/s]

Saved: final_results\WFG.npz
Problem set: RW. 10 total problems


  0%|          | 0/120 [00:00<?, ?it/s]

Saved: final_results\RW.npz
Problem set: WFG. 27 total problems


  0%|          | 0/324 [00:00<?, ?it/s]

Saved: final_results\WFG_HD.npz


### Perform statistical comparisons for each set of problems
Note that this all for each indicator

- Per scalarisation:
    - model is best per # of dimensions
    - model is best per # of objectives
- Which scalarisation and model combination is overall the best

In [45]:
# base settings
time = -1  # time index at which to compare methods [default: last idx (-1)]
n_runs = 21

# indicators and the direction of their stats tests, e.g., we want to 
# minimise IGD+ and maximise hypervolume (HV)
indicators_and_args = [
    ("igd+", np.argmin, "less"),
    ("hv", np.argmax, "greater"),
]

n_methods = len(model_dict) # 3

#### Which model is best per number of dimensions (for each scalarisation)

In [ ]:
for original_problem_name in tqdm.notebook.tqdm(problem_names):

    problem_name, prob_dict = mbore.problem_sets.get_problem_dict(
        original_problem_name
    )
    total_problems = mbore.problem_sets.get_number_of_problems(prob_dict)

    # load the results file
    final_results_path = os.path.join(
        final_results_dir, f"{original_problem_name:s}.npz"
    )

    # D[problem_id][dim][fdim][scalarizer][model][indicator]
    # contains an np.ndarray of shape (n_runs, budget)
    with np.load(final_results_path, allow_pickle=True) as fd:
        assert fd["original_problem_name"] == original_problem_name
        D = fd["D"].item()

    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    # model (i.e. GP, MLP, XGB) is best per problem, split by problem dims or objs
    td = {}

    with tqdm.notebook.tqdm(
        total=total_problems * len(scalarizers) * len(indicators_and_args), 
        leave=False
    ) as pbar:
        for (indicator, argmethod, wilcoxon_side) in indicators_and_args:
            td[indicator] = {}

            for scalarizer in scalarizers:
                td[indicator][scalarizer] = {}

                # populate the data we wish to process
                for problem_id in prob_dict:
                    td[indicator][scalarizer][problem_id] = {}

                    for dim, fdims in prob_dict[problem_id]:
                        td[indicator][scalarizer][problem_id][dim] = {}

                        for fdim in fdims:
                            # storage
                            best_seen_values = np.zeros((n_methods, n_runs))

                            # extract the best seen values for each method
                            for i, model in enumerate(model_dict):
                                best_seen_values[i, :] = D[
                                    problem_id
                                ][dim][fdim][scalarizer][model][indicator][:, time]

                            # do the stats testing -- one-sided wilcoxon
                            # signed-rank test with holm-bonferroni correction
                            (medians, mads, best_mask) = mbore.util.stats_test(
                                best_seen_values, argmethod, wilcoxon_side
                            )

                            td[indicator][scalarizer][problem_id][dim][fdim] = {
                                "medians": medians,
                                "mads": mads,
                                "best_mask": best_mask,
                            }

                            pbar.update()

    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    # which scalariser and model combo is best? i.e., which should we pick?
    # this table just stores the median fitness values and the
    # best values (or stats equal to best) values as a mask
    # tds[indicator][scalerizer][problem_id][dim][fdim]
    tds = {}

    # gather up the keys for this list
    n_scalarizers_and_models = len(scalarizers) * len(model_dict)

    scalarizers_and_model_combos = [
        (scalarizer, model)
        for scalarizer in scalarizers
        for model in model_dict
    ]

    with tqdm.notebook.tqdm(
        total=len(indicators_and_args) * total_problems,
        leave=False
) as pbar:
        for (indicator, argmethod, wilcoxon_side) in indicators_and_args:
            tds[indicator] = {}

            # populate the data we wish to process
            for problem_id in prob_dict:
                tds[indicator][problem_id] = {}

                for dim, fdims in prob_dict[problem_id]:
                    tds[indicator][problem_id][dim] = {}

                    for fdim in fdims:

                        best_seen_values = np.zeros(
                            (n_scalarizers_and_models, n_runs)
                        )

                        for i, (scalarizer, model) in enumerate(
                            scalarizers_and_model_combos
                        ):
                            # extract the best seen values for each method
                            best_seen_values[i, :] = D[problem_id][dim][fdim][
                                scalarizer
                            ][model][indicator][:, time]

                        # do the stats testing -- one-sided wilcoxon
                        # signed-rank test with holm-bonferroni correction
                        (medians, mads, best_mask) = mbore.util.stats_test(
                            best_seen_values, argmethod, wilcoxon_side
                        )

                        tds[indicator][problem_id][dim][fdim] = {
                            "medians": medians,
                            "mads": mads,
                            "best_mask": best_mask,
                        }

                        pbar.update()

    savename = f"{original_problem_name:s}_statstests.npz"
    savepath = os.path.join(final_results_dir, savename)
    np.savez(
        savepath, 
        td=td, 
        tds=tds, 
        original_problem_name=original_problem_name
    )